In [67]:
import numpy as np
import torch

def load_array(filename, task):
    datapoint = np.load(filename)
    if task == 'task 1':
        initial_state = datapoint['initial_state']
        terminal_state = datapoint['terminal_state']
        return initial_state, terminal_state
    elif task == 'task 2' or task == 'task 3':
        whole_trajectory = datapoint['trajectory']
        # change shape: (num_bodies, attributes, time) ->  num_bodies, time, attributes
        whole_trajectory = np.swapaxes(whole_trajectory, 1, 2)
        initial_state = whole_trajectory[:, 0]
        target = whole_trajectory[:, 1:, 1:]  # drop the first timepoint (second dim) and mass (last dim) for the prediction task
        return initial_state, target
    else:
        raise NotImplementedError("'task' argument should be 'task 1', 'task 2' or 'task 3'!")

In [68]:
#### Create adjacency matrix

# Define distance metrics
def euclidean_distance(x, y):
    return torch.sqrt(torch.sum((x - y)**2))

def inverse_distance(x, y):
    return 1 / euclidean_distance(x, y)

# Create adjacency matrix function
def create_adjacency_matrix(data, distance_metric):
    n = data.shape[0]
    adjacency_matrix = torch.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i != j:  # we don't calculate the distance of the object to itself
                # we extract the position [x, y] for both objects i and j
                position_i = data[i, 1:3]
                position_j = data[j, 1:3]
                adjacency_matrix[i, j] = distance_metric(position_i, position_j)
    return adjacency_matrix

# Validate input
def validate_input(X, adjacency_matrix):
    # X should be a 2D tensor
    assert X.dim() == 2, f"X must be 2D, but got shape {X.shape}"

    # The number of nodes should be the same in X and the adjacency matrix
    assert X.shape[0] == adjacency_matrix.shape[0] == adjacency_matrix.shape[1], \
        f"Mismatch in number of nodes: got {X.shape[0]} nodes in X, but {adjacency_matrix.shape[0]} nodes in adjacency matrix"

    # The adjacency matrix should be square
    assert adjacency_matrix.shape[0] == adjacency_matrix.shape[1], \
        f"Adjacency matrix must be square, but got shape {adjacency_matrix.shape}"

    print("All checks passed.")


In [69]:
# Load and examine sample data
"""
This cell gives an example of loading a datapoint with numpy for task 1.

The arrays returned by the function are structures as follows:
initial_state(X): shape (n_bodies, [mass, x, y, v_x, v_y])
terminal_state(y): shape (n_bodies, [x, y])

"""


X, y= load_array('data/task 1/train/trajectory_0.npz', task='task 1')
X = torch.tensor(X, dtype=torch.float32)


In [18]:
adjacency_matrix = create_adjacency_matrix(X, inverse_distance)

In [70]:
# Test network
import torch
from torch_geometric.nn import GraphConv
from torch_geometric.data import Data

class SimpleGNN(torch.nn.Module):
    def __init__(self, num_features, hidden_channels, num_classes):
        super(SimpleGNN, self).__init__()
        self.conv1 = GraphConv(num_features, hidden_channels)
        self.conv2 = GraphConv(hidden_channels, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # 1st Graph Convolution layer
        x = self.conv1(x, edge_index)
        x = torch.relu(x)

        # 2nd Graph Convolution layer
        x = self.conv2(x, edge_index)

        return x


# Convert adjacency matrix to edge_index
# WARNING! This will 'Delete' the distances between the nodes
edge_index = adjacency_matrix.nonzero().t()

# Create a Data object
data = Data(x=X, edge_index=edge_index)

# Create an instance of our GNN
model = SimpleGNN(num_features=5, hidden_channels=32, num_classes=2)

# Pass the graph through the model
out = model(data)

print(out.shape)
edge_index, adjacency_matrix

torch.Size([8, 2])


(tensor([[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3,
          3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6,
          6, 7, 7, 7, 7, 7, 7, 7],
         [1, 2, 3, 4, 5, 6, 7, 0, 2, 3, 4, 5, 6, 7, 0, 1, 3, 4, 5, 6, 7, 0, 1, 2,
          4, 5, 6, 7, 0, 1, 2, 3, 5, 6, 7, 0, 1, 2, 3, 4, 6, 7, 0, 1, 2, 3, 4, 5,
          7, 0, 1, 2, 3, 4, 5, 6]]),
 tensor([[0.0000, 0.0918, 3.3833, 0.3152, 0.2411, 0.1777, 0.0905, 0.1391],
         [0.0918, 0.0000, 0.0915, 0.0879, 0.1365, 0.1583, 0.0630, 0.2146],
         [3.3833, 0.0915, 0.0000, 0.2885, 0.2323, 0.1807, 0.0930, 0.1402],
         [0.3152, 0.0879, 0.2885, 0.0000, 0.2457, 0.1319, 0.0703, 0.1143],
         [0.2411, 0.1365, 0.2323, 0.2457, 0.0000, 0.2297, 0.0743, 0.1993],
         [0.1777, 0.1583, 0.1807, 0.1319, 0.2297, 0.0000, 0.0975, 0.5669],
         [0.0905, 0.0630, 0.0930, 0.0703, 0.0743, 0.0975, 0.0000, 0.0874],
         [0.1391, 0.2146, 0.1402, 0.1143, 0.1993, 0.5669, 0.0874, 0.0000]])

In [59]:
# Model Zoo

from torch_geometric.nn import SAGEConv

class GraphSAGE(torch.nn.Module):
    def __init__(self, num_features, hidden_channels, num_classes):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(num_features, hidden_channels*2)
        self.conv2 = SAGEConv(hidden_channels*2, hidden_channels)
        self.conv3 = SAGEConv(hidden_channels, num_classes)
        self.dropout = torch.nn.Dropout(p=0.3)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # 1st GraphSAGE layer
        x = self.conv1(x, edge_index)
        x = torch.relu(x)

        # 2nd GraphSAGE layer
        x = self.conv2(x, edge_index)
        x = torch.relu(x)
        # x = self.dropout(x)
        # 3rd GraphSAGE layer
        x = self.conv3(x, edge_index)

        return x

In [53]:
# DataLoader

from torch_geometric.data import Dataset, Data, DataLoader

class MyDataset(Dataset):
    def __init__(self, root, filenames, transform=None, pre_transform=None):
        self.filenames = filenames
        super(MyDataset, self).__init__(root, transform, pre_transform)

    @property
    def raw_file_names(self):
        return self.filenames

    def len(self):
        return len(self.filenames)

    def get(self, idx):
        X, y = load_array(self.filenames[idx], task='task 1')
        X = torch.tensor(X, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32)

        adjacency_matrix = create_adjacency_matrix(X, inverse_distance)
        edge_index = adjacency_matrix.nonzero().t()

        data = Data(x=X, y=y, edge_index=edge_index)

        return data

filenames = [f'data/task 1/train/trajectory_{i}.npz' for i in range(900)]
dataset = MyDataset(root='data/task 1/train', filenames=filenames)

# Prepare for validation data set

val_filenames = [f'data/task 1/test/trajectory_{i}.npz' for i in range(901, 1000)]
val_dataset = MyDataset(root='data/task 1/test', filenames=val_filenames)
val_dataloader = DataLoader(val_dataset, batch_size=32)

dataloader = DataLoader(dataset, batch_size=32)

In [60]:
device = "mps"
model = GraphSAGE(num_features=5, hidden_channels=16, num_classes=2).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.MSELoss()  # we use Mean Squared Error loss for regression tasks


for epoch in range(100):  # run for 100 epochs
    # Training
    model.train()
    for batch in dataloader:
        batch = batch.to(device)  # move batch to the device
        optimizer.zero_grad()  # set gradients to zero
        out = model(batch)  # forward pass
        loss = criterion(out, batch.y)  # compute loss
        loss.backward()  # backward pass (compute gradients)
        optimizer.step()  # update model parameters

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            batch = batch.to(device)
            out = model(batch)
            val_loss += criterion(out, batch.y).item() * batch.num_graphs

    val_loss /= len(val_dataset)  # compute average validation loss

    print(f'Epoch: {epoch+1}, Training Loss: {loss.item()}, Test Loss: {val_loss}')



Epoch: 1, Training Loss: 19.18354034423828, Test Loss: 15.109752529799335
Epoch: 2, Training Loss: 12.629542350769043, Test Loss: 8.984456611402107
Epoch: 3, Training Loss: 8.390835762023926, Test Loss: 5.331993970003995
Epoch: 4, Training Loss: 7.516866207122803, Test Loss: 5.008034535128661
Epoch: 5, Training Loss: 7.09335470199585, Test Loss: 4.8860001636273935
Epoch: 6, Training Loss: 6.73557710647583, Test Loss: 4.799153523011641
Epoch: 7, Training Loss: 6.436294078826904, Test Loss: 4.731423467096656
Epoch: 8, Training Loss: 6.156905651092529, Test Loss: 4.674902643820252
Epoch: 9, Training Loss: 5.91514253616333, Test Loss: 4.628289037280613
Epoch: 10, Training Loss: 5.67647647857666, Test Loss: 4.598819756748701
Epoch: 11, Training Loss: 5.492270469665527, Test Loss: 4.579380160630351
Epoch: 12, Training Loss: 5.3515448570251465, Test Loss: 4.565064478402186
Epoch: 13, Training Loss: 5.210579872131348, Test Loss: 4.552286569518272
Epoch: 14, Training Loss: 5.0843987464904785, T

In [66]:
# Calculate static and linear baselines (formula on ANS)
# THIS IS CORRECT! DO NOT CHANGE!

def static_baseline(X):
    return X[:, 1:3]  # initial x,y coordinates

def linear_baseline(X):
    return X[:, 1:3] + X[:, 3:5] * 5  # initial x,y coordinates plus velocity times time

def compute_baseline_loss(baseline_fn, dataloader):
    total_loss = 0
    criterion = torch.nn.MSELoss()  # we use Mean Squared Error loss for regression tasks

    for batch in dataloader:
        batch = batch.to(device)
        predictions = baseline_fn(batch.x).to(device)
        total_loss += criterion(predictions, batch.y).item() * batch.num_graphs

    return total_loss / len(dataloader.dataset)

train_loss_static = compute_baseline_loss(static_baseline, dataloader)
train_loss_linear = compute_baseline_loss(linear_baseline, dataloader)

val_loss_static = compute_baseline_loss(static_baseline, val_dataloader)
val_loss_linear = compute_baseline_loss(linear_baseline, val_dataloader)


 # now print out with filler spaces to make it easier to read
print(f'Training Loss   - Static Baseline: {train_loss_static:0.4f}, Linear Baseline: {train_loss_linear:0.4f}')
print(f'Validation Loss - Static Baseline: {val_loss_static:0.4f}, Linear Baseline: {val_loss_linear:0.4f}')


Training Loss   - Static Baseline: 13.9304, Linear Baseline: 22.1829
Validation Loss - Static Baseline: 11.4824, Linear Baseline: 20.2710
